# Osa 6: Käyttäjien hallinta
Tässä osassa tutustutaan erilaisiin käyttäjärooleihin ja niiden hallintaan.

In [ ]:
# Jos et ole suorittanut kaikkia aiempia harjoituksia, aja tämä solu
from utils import initializer
initializer.initialize_h7()

### Käyttäjäroolit
PostgreSQL:ssä käyttäjiä hallitaan rooleina (roles). Rooleja, joilla kirjaudutaan tietokantaan, kutsutaan nimellä *login roles*. Rooleja, jotka sisältävät muita rooleja kutsutaan nimellä *group roles*.

*Group role* ryhmiin voidaan myös määrittää kirjautumisoikeudet tietokantaan, mutta tietokannan ylläpito
on helpompi jos sitä vältetään.

> Aiemmissa PostgreSQLversioissa käytetiin termejä users ja group, edelleen niitä voi käyttää mutta kannattaa käyttää vastaavia uusia termejä, role ja group role.


Oletuksena PostgreSQL asennuksessa on luotu *postgres* nimiset rooli ja tietokanta. Sen lisäksi aiemmin
luotiin koulutus niminen tietokanta. Luo nyt uusi rooli, jonka nimi on *matti*:

In [ ]:
# Lataa SQL-lisäosa
%load_ext sql
# Luo tietokantayhteys tietokantaan koulutus
from ipygis import get_connection_url
c_url = get_connection_url(dbname='koulutus')
%sql $c_url

In [ ]:
%%sql

CREATE ROLE matti LOGIN
PASSWORD '1234'
CREATEDB
VALID UNTIL 'infinity';

CREATEDB valitsin määrittää oikeudet roolille luoda tietokantoja.

VALID määrittää kuinka pitkään rooli on voimassa.

Uusi matti rooli on voimassa toistaiseksi. Sillä on oikeudet tietokantojen luomiseen, muttei ole määritelty
ylläpitäjäksi, eli sillä on rajoitetut oikeudet.

Luo nyt rooli, jolla on ylläpitäjän oikeudet käyttämällä SUPERUSER valitsinta:

In [ ]:
%%sql

CREATE ROLE yp LOGIN
PASSWORD '1234'
SUPERUSER
VALID UNTIL '2021-1-1 00:00';

Uusi yp-roolilla on ylläpidon oikeudet ja sen salasana on voimaassa 1. tammikuun 2021 asti.
Kun haluat nähdä palvelimessa olemassa roolit, käytä seuraavaa [psql](H2_tyokalut.ipynb#psql)-komentoa: `\du`.

Luodaan vielä kolmas käyttäjä `maija`:

In [ ]:
%sql CREATE ROLE maija LOGIN PASSWORD '1234';

## Käyttöoikeuksien lisääminen
Käyttöoikeuksia voi lisätä `GRANT`-komennolla ja vastaavasti niitä voi poistaa `REVOKE`-komennolla. Kokeile matti-käyttäjänä lukea taulua etunimet:



In [ ]:
c_url = get_connection_url(user='matti', password='1234', dbname='koulutus')
%sql $c_url
%sql SELECT * FROM etunimet LIMIT 10;

Lukeminen ei onnistunut käyttöoikeuksien puutteen takia. Lisää postgres-käyttäjänä matille käyttöoikeus lukea taulua `etunimet` ja myös oikeus luoda uusia skeemoja ja tauluja

In [ ]:
c_url = get_connection_url(user='postgres', dbname='koulutus')
%sql $c_url
%sql GRANT SELECT ON etunimet TO matti;
%sql GRANT CREATE ON DATABASE koulutus TO matti

Nyt lukemisen pitäisi onnistua

In [ ]:
# Luo tietokantayhteys tietokantaan koulutus käyttäjällä matti
c_url = get_connection_url(user='matti', password='1234', dbname='koulutus')
%sql $c_url
%sql SELECT * FROM etunimet LIMIT 10;

Luo nyt matti-käyttäjänä skeema `matin_skeema` ja sinne taulu `matit`. Poista samalla publicilta oikeudet tauluun.

In [ ]:
%%sql
CREATE SCHEMA matin_skeema;
CREATE TABLE matin_skeema.matit AS (SELECT * FROM etunimet WHERE nimi LIKE 'Mat%');
REVOKE ALL ON TABLE matin_skeema.matit FROM public;

Yritä nyt käyttäjänä *maija* lukea taulua matit:

In [ ]:
c_url = get_connection_url(user='maija', password='1234', dbname='koulutus')
%sql $c_url
%sql SELECT * FROM matin_skeema.matit LIMIT 10;

## **Harjoitus 6.1**: käyttöoikeudet
Lisää käyttäjälle *maija* käyttöoikeudet lukea ja kirjoittaa taulua `matin_skeema.matit`

In [ ]:
# Tämä solu ajetaan postgres-käyttäjänä
c_url = get_connection_url(dbname='koulutus')
%sql $c_url
# Luo maijalle oikeus lukea taulua etunimet
%sql GRANT SELECT ...
# Anna maijalle oikeus käyttää skeemaa matin_skeema
%sql GRANT USAGE ON SCHEMA ...
# Anna maijalle luku- ja kirjoitusoikeus tauluun matin_skeema.matit
%sql GRANT SELECT, INSERT ...

In [ ]:
# Tämä solu ajetaan postgres-käyttäjänä
c_url = get_connection_url(dbname='koulutus')
%sql $c_url
%sql GRANT SELECT ON etunimet TO maija;
%sql GRANT USAGE ON SCHEMA matin_skeema TO maija;
%sql GRANT SELECT, INSERT ON matin_skeema.matit TO maija

Lisää samalla tauluun myös etunimet, jotka alkavat merkkijonolla "Maij".

In [ ]:
# Tämä solu ajetaan maija-käyttäjänä
c_url = get_connection_url(user='maija', password='1234', dbname='koulutus')
%sql $c_url
%sql INSERT ...

In [ ]:
# Tämä solu ajetaan maija-käyttäjänä
c_url = get_connection_url(user='maija', password='1234', dbname='koulutus')
%sql $c_url
%sql INSERT INTO matin_skeema.matit (SELECT * FROM etunimet WHERE nimi LIKE 'Maij%');

### Tarkistus
Tarkista, että sait lisättyä Maijalle käyttöoikeudet.

In [ ]:
res = %sql SELECT * FROM matin_skeema.matit WHERE nimi LIKE 'Maij%'
assert len(res) == 43, f'Tarkista tehtävä, saatiin {len(res)} riviä tulokseksi'
res

## Ryhmäroolit
`Group role` luodaan SQL komennolla:
```sq
CREATE ROLE roolin_nimi INHERIT;
```

In [ ]:
c_url = get_connection_url(dbname='koulutus')
%sql $c_url
%sql CREATE ROLE editor INHERIT;

INHERIT valitsin tarkoittaa se, että editor rooli perii oikeudet niiltä rooleilta, joiden jäsen se on. Poikeuksena superuser oikeus, joka ei kuitenkaan koskaan periydy PostgreSQL:ssä.

Lisää äsken luomasi roolit ryhmään seuraavasti:

In [ ]:
%%sql
GRANT editor TO matti;
GRANT editor TO yp;

## Roolien käyttöä

Tähän asti olemme muodostanut yhteyttä palvelimeen postgres-roolilla, jolla muun muuassa on
ylläpitäjän oikeudet. Salasana on tallennettu ympäristömuuttujaan, josta psql osaa sen kaivaa. Psql:n saa kuitenkin kysymään salasanaa antamalla sille argumentin **W**. Kirjautuminen tapahtuu konsolista tällä tutulla komennolla:

```sh
psql -U matti -W
```
Kirjautuessa pyydetään salasana, sen määriteltiin aiemmin: 1234. Mutta kirjautuminen kuitenkin
epäonnistu, koska palvelimesta ei löytynyt matti-nimistä tietokantaa.

PostgreSQL vaatii, että kirjaudutaan johonkin tietokantaan. Tähän asti ei tämä ole ollut ongelmaa koska,
kirjauduttiin PostgreSQL postgres-roolilla ja saman nimisen tietokanta on kyllä olemassa palvelimessa.

Kun kirjaudutaan PostgreSQL, täytyy määritellä tietokannan nimi. Esimerkiksi:

```sh
psql -U matti -d koulutus -W
```

Huomaa vielä, että nyt psql-konsolissa tietokannan nimen jälkeen merkki on nyt > eikä #. Tämä
tarkoittaa, että käyttäjällä ei ole ylläpidon oikeuksia.

## **Harjoitus 6.2**: Roolien luominen pgAdmin kautta
Roolien hallinta pgAdmin käyttöliittymän kautta on huomattavasti selkeämpää.

<img src="img/pgadmin_create_role.gif" alt="createrole_pgadmin" style="width: 700px;"/>
Luo vielä yksi uusi rooli painamalla oikea hiiren näppäintä Login roles kohtaa ja täytä tiedot eri
välilehdissä seuraavaksi:

    nimi: tiia
    salasan: 1234
    rooliryhmä: editor
    ylläpitäjän oikeudet

Tutustuu muihin asetuksia ja kokeile niitä uudessa roolissa.

Kokeile myös poistaa yp-rooli käyttöliittymän kautta.
> SQL-komento käyttäjien poistamiseen on
    ```
    DROP ROLE <nimi>;
    ```

## **Harjoitus 6.3**: Tietokannan omistussuhteen muutos
Edellisissä harjoituksessa luotiin tietokanta koulutus. Tietokannan omistaja on postgres.

Vaihdetaan nyt omistus matti-roolille.
Muodosta konsolilla uutta yhteyttä tietokantaan postgres-roolilla, sillä taulun omistajan vaihdon voi suorittaa vain ylläpitäjä tai taulun omistaja. Käytä syntaksia:

```sql
ALTER DATABASE tietokanta OWNER TO käyttäjä;
```

In [ ]:
c_url = get_connection_url() # ilman parametreja postgres-käyttäjä
%sql $c_url

In [ ]:
%sql ALTER ...

In [ ]:
%%sql
ALTER DATABASE koulutus OWNER TO matti;

### Tarkistus
Aja alla olevat kaksi solua tarkistaaksesi, että omistuksen muutos onnistui.

In [ ]:
%%sql
res <<
SELECT d.datname as "Name", pg_catalog.pg_get_userbyid(d.datdba) as "Owner"
    FROM pg_catalog.pg_database d
    WHERE d.datname = 'koulutus'
ORDER BY 1;

In [ ]:
assert res[0][1] == 'matti', f'Omistaja on matin sijaan {res[0][1]}'
res

<a rel="license" href="http://creativecommons.org/licenses/by-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nd/4.0/88x31.png" />